<a href="https://colab.research.google.com/github/e-olang/NLP/blob/main/OpenAi/Whisper_ASR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# installs

!pip3 install pytube -q         # any other alt, wiill work, chosse what works best for you
!pip install git+https://github.com/openai/whisper.git -q
!pip install gradio -q

     |████████████████████████████████| 56 kB 4.4 MB/s 
     |████████████████████████████████| 4.9 MB 20.0 MB/s 
     |████████████████████████████████| 120 kB 68.8 MB/s 
     |████████████████████████████████| 6.6 MB 62.6 MB/s 
     |████████████████████████████████| 5.3 MB 30.7 MB/s 
     |████████████████████████████████| 84 kB 3.9 MB/s 
     |████████████████████████████████| 54 kB 1.1 MB/s 
     |████████████████████████████████| 55 kB 3.3 MB/s 
     |████████████████████████████████| 57 kB 5.6 MB/s 
     |████████████████████████████████| 212 kB 73.1 MB/s 
     |████████████████████████████████| 270 kB 55.3 MB/s 
     |████████████████████████████████| 2.3 MB 47.2 MB/s 
     |████████████████████████████████| 84 kB 3.5 MB/s 
     |████████████████████████████████| 112 kB 74.8 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 80 kB 8.9 MB/s 
     |████████████████████████████████| 68 kB 7.1 MB/s 
     |██████████████████████████

In [2]:
import whisper
from pytube import YouTube
import os
import gradio as gr

In [3]:
def yt_audio(link):
	yt = YouTube(str(link))
	video = yt.streams.filter(only_audio=True).first()
	destination = '.'
	audio_file = video.download(output_path=destination)
	base, ext = os.path.splitext(audio_file)
	new_file = base + '.mp3'
	os.rename(audio_file, new_file)
	
	return str(new_file)

In [4]:
#model = whisper.load_model('base')
model = whisper.load_model('small')

100%|███████████████████████████████████████| 461M/461M [00:07<00:00, 66.8MiB/s]


In [13]:
sample_file = yt_audio('https://www.youtube.com/watch?v=9V3L8R5DlNg')

In [14]:
sample_file

'/content/./Twitter How many bot accounts are there - BBC News.mp3'

In [15]:
sample_transcription = model.transcribe(sample_file)

In [22]:
print(sample_transcription["language"])

en


In [19]:
type(sample_transcription)

dict

In [21]:
print(sample_transcription.keys())

dict_keys(['text', 'segments', 'language'])


In [20]:
import json

with open("sample.json", "w") as outfile:
    json.dump(sample_transcription, outfile)

In [16]:
from whisper.utils import write_vtt

with open('sub.txt', "w") as txt:
    write_vtt(sample_transcription["segments"], file = txt)

In [17]:
with open('sub.txt', "r") as txt:
    print('\n'.join(txt.readlines()))

WEBVTT



00:00.000 --> 00:06.800

 Twitter says that it has nearly 240 million monetizable daily active users.



00:06.800 --> 00:09.640

 Millions of those accounts though aren't real.



00:09.640 --> 00:12.160

 They're bots, spam and fake accounts.



00:12.160 --> 00:15.400

 And that's something both Twitter and Elon Musk accept.



00:15.400 --> 00:20.280

 But what they don't agree on is how many accounts are fake, or even what a bot is.



00:20.280 --> 00:27.520

 Depending on how you define bot, you could have anywhere from less than 1% to 20%.



00:27.520 --> 00:35.520

 It's a debate that's turned into a $44 billion question as Elon Musk attempts to back out of a deal to buy Twitter.



00:35.520 --> 00:39.320

 So how many bots are there on Twitter?



00:39.320 --> 00:44.400

 First of all, Twitter claims that less than 5% of all of its daily active users are bots.



00:44.400 --> 00:47.200

 Elon Musk though says that figure is nonsense.



00:47.200 --> 00:48.440



------

In [ ]:
def file_transcribe(audio):
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    _, probs = model.detect_language(mel)
    #print(f"Detected Language:  {max(probs, key=probs.get)}")
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    #print(result.text)
    return result.text

In [ ]:
def link_transcribe(link):
    audio_path = yt_audio(link)
    audio = whisper.load_audio(audio_path)
    audio = whisper.pad_or_trim(audio)

    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    _, probs = model.detect_language(mel)
    #print(f"Detected Language:  {max(probs, key=probs.get)}")
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    #print(result.text)
    return result.text

In [ ]:
link_transcribe("https://youtu.be/H38HTWLwgDs")

"I'm good, I'm feeling alright baby, I'ma have the best fucking night of my life"

In [ ]:
gr.Interface(
    title = 'OpenAi Whisper',
    fn = link_transcribe,
    inputs = ['textbox'],
    outputs = ['textbox'],
    live = True).launch()



"""
change input to: '[
        gr.inputs.Audio(source="microphone", type = "filepath")
    ],'
to use microphone audio, rem to also change transcription fucntion to 'file_trascribe'
"""